### This code solves the background equations of motion of arXiv:2008.13660
### --- Background_engine.py performs the main calculations.
### --- model.py contains the definition of the model.
### --- Spinning_Quintessence.ipynb is the wrapper notebook.
### --- Plotting.ipynb produces the plots in the paper.







In [4]:
#Some imports used later

import numpy as np
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from scipy.optimize import minimize_scalar

from Background_engine import *


N_lst = np.linspace(-15., 2., 1e5)
h = N_lst[1] - N_lst[0]

In [5]:
Omega_m = 0.3
Omega_r = 6e-5


alpha = 2e-3 # In units of Mpl^2H_0^2
m = 30. # In units of H_0
vev = 7.*1e-4 # In units of M_pl
V0 = 2. # In units of Mpl^2H_0^2. This value will later be adjusted to have a flat universe.


### --- Select the model here --- ###
model = "tilted_higgs_like" #tilted_higgs_like_exp"

### --- Specify the initial conidtions --- ###
### The values of velocities are not important, 
### but should be set to non-zero values. 
### Otherwise Omega is ill-definied initially! 
r_ini = 1.*vev
theta_ini = 0.
initial_condition = [r_ini, 1e-10, theta_ini, 1e-10]

In [6]:
### --- This ugly function connect to the solver engine 
### and gets the required solutions for the given value of m

def with_matter(m):
    
    ### --- This function iterates over V_0 in order to 
    ### maintain a spatially flat universe.
    
    def E_today(V0_tmp):
    

        params = [m, vev, V0_tmp, alpha, model, Omega_m, Omega_r]
        soln = odeint(diff_evolve, initial_condition, N_lst, args=(params,))
    
        phi_vec = np.ones(2)
        velo_vec = np.ones(2)
    
        r_interp = interpolate.interp1d(N_lst, soln[:,0])
        phi_vec[0] = r_interp(0.)
        theta_interp = interpolate.interp1d(N_lst, soln[:,2])
        phi_vec[1] = theta_interp(0.)
    
        r_prime_interp = interpolate.interp1d(N_lst, soln[:,1])
        velo_vec[0] = r_prime_interp(0.)
        theta_prime_interp = interpolate.interp1d(N_lst, soln[:,3])
        velo_vec[1] = theta_prime_interp(0.)
    

        sol_Hubble_E_sqr = Hubble_func_sqr(phi_vec, velo_vec, 0., params)

    
        return np.abs(np.sqrt(sol_Hubble_E_sqr) - 1.)
    
    V0_finding = minimize_scalar(E_today, bounds=(1., 10.), method='bounded')
    V0 = V0_finding.x
    print("V0 = ", V0, "E_0 = ", E_today(V0) + 1.)

    params = [m, vev, V0, alpha, model, Omega_m, Omega_r]
    soln = odeint(diff_evolve, initial_condition, N_lst, args=(params,))
    
    ### This is just to make sure that Eq. 35 is satisfied.
    tmp = np.sqrt(params[0]*np.sqrt(3.*params[2])/params[3])
    print(1., params[1]*tmp/4., params[2]/6./params[0]**2, 1./tmp**2/4.)


    ### --- These are the solutions
    r = soln[:, 0]
    r_prime = soln[:, 1]
    theta = soln[:, 2]
    theta_prime = soln[:, 3]

    ### --- These are various useful quantities
    sol_eps = np.ones(len(N_lst))
    epsilon_potential = np.ones(len(N_lst))
    turning_term = np.ones(len(N_lst))
    eps_V_plt = np.ones(len(N_lst))
    Hubble_sqr_plt = np.ones(len(N_lst))
    pot_plt = np.ones(len(N_lst))

    for indx, N_tmp in enumerate(N_lst):
        phi_vec = np.array([r[indx], theta[indx]])
        velo_vec = np.array([r_prime[indx], theta_prime[indx]])
    
        sol_eps[indx] = eps(phi_vec, velo_vec, N_tmp, params)
        epsilon_potential[indx] = Omega_term(phi_vec, velo_vec, N_tmp, params)[1]
        turning_term[indx] = Omega_term(phi_vec, velo_vec, N_tmp, params)[0]
        eps_V_plt[indx] = eps_V(phi_vec, params)
        Hubble_sqr_plt[indx] = Hubble_func_sqr(phi_vec, velo_vec, N_tmp, params)
        pot_plt[indx] = pot(phi_vec, params)
    
    
    scalar_pressure = (r_prime**2 + theta_prime**2*r**2)*Hubble_sqr_plt/2. - pot_plt
    scalar_energy = (r_prime**2 + theta_prime**2*r**2)*Hubble_sqr_plt/2. + pot_plt
    
    ### The output is encoded in this horrendous list. A dict would have been nicer, but whatever.
    out = [r, r_prime, theta, theta_prime, Hubble_sqr_plt, scalar_pressure, scalar_energy, \
            alpha, m, vev, V0, turning_term, sol_eps, epsilon_potential]
    return out

In [7]:
### --- Here we just do the calculation for a bunch of m values, 
### and dump the results to files.

for m_indx, m_tmp in enumerate(np.linspace(30., 80., 6)):
    
    out = with_matter(m_tmp)
    print(m_tmp, " -> done!")
    
    print("alpha -> ", out[7])
    print("m -> ", out[8])
    print("vev -> ", out[9])
    print("V0 -> ", out[10])
    print("######---#########---######")
    
    file_name = 'Files/with_matter_m_' + str(m_indx + 1) + '.npz'
    np.savez(file_name, out)

V0 =  2.1373052891680278 E_0 =  1.0000001370264702
1.0 0.03410598777242235 0.000395797275771857 6.5819539884165905e-06
30.0  -> done!
alpha ->  0.002
m ->  30.0
vev ->  0.0007
V0 ->  2.1373052891680278
######---#########---######
V0 =  2.1475631282403973 E_0 =  1.0000000951782626
1.0 0.039429370628743174 0.0002237044925250414 4.924661861360165e-06
40.0  -> done!
alpha ->  0.002
m ->  40.0
vev ->  0.0007
V0 ->  2.1475631282403973
######---#########---######
V0 =  2.1575986347087515 E_0 =  1.0000001219994965
1.0 0.04413478663323809 0.00014383990898058342 3.93055649889845e-06
50.0  -> done!
alpha ->  0.002
m ->  50.0
vev ->  0.0007
V0 ->  2.1575986347087515
######---#########---######
V0 =  2.1673965093612377 E_0 =  1.0000000687005905
1.0 0.04840203062609868 0.00010034243098894619 3.268051876235309e-06
60.0  -> done!
alpha ->  0.002
m ->  60.0
vev ->  0.0007
V0 ->  2.1673965093612377
######---#########---######
V0 =  2.1769533213808216 E_0 =  1.0000002444708813
1.0 0.05233770364954217 7.4